In [ ]:
import random
import shutil
from shutil import copyfile
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_dir  = '/content/drive/compVisionPackage/Agricultural-crops'
os.chdir(base_dir)

# list each directory name with # (label name)
drive_list = tf.io.gfile.listdir(base_dir)

# display number of labels
drive_labels = len(drive_list)
print(f"Total labels: = {drive_labels}")

Total Class Labels = 30


In [ ]:
# Conversion of images into NumPy arrays
def convert_jpeg_to_csv(img):
  arr = np.asarray(img)
  print(arr.shape)
# Make a 2D list of lists from a 3D array
  lst = []
  for row in arr:
    tmp = []
    for col in row:
        tmp.append(str(col))
    lst.append(tmp)
#Copy a list to a CSV file.
  with open('my_file.csv', 'w') as f:
    for row in lst:
        f.write(','.join(row) + '\n')

In [ ]:
def svm(W, X, Y):
    reg_strength = 10000  # regularization strength

    #  compute svm
    N = X.shape[0]
    # np.dot(X, W) product of two arrays
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = reg_strength * (np.sum(distances) / N)

    # determine costs
    # np.dot(W, W) product of two arrays
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

In [ ]:
# EX2.Divide the data into three sets: test, validation, and train.


import numpy as np  # for handling multi-dimensional array operation
import pandas as pd  # for reading data from csv
from sklearn.model_selection import train_test_split as tts #Split arrays or matrices into random train and test subsets


learning_rate = 0.000001

# main method where we train our dataset
def init():
    
    # SVM only takes numbers as input.This indicates that we ought to convert our jpg to csv.
    # TODO:We should convert jpg to csv prior to pd.read_csv.
    for file in os.listdir(base_dir): #throw every directory
      if file.endswith(".jpeg"): #find files ending with .jpeg
        convert_jpeg_to_csv(file) #call function to cinvert jpeg to csv


    data = pd.read_csv('./data.csv')
    # Consequently, the M and B categories will be transformed into
    # values 1 and -1 (or -1 and 1), respectively.
    collect_map = {'M': 1, 'B': -1}
    data['diagnosis'] = data['diagnosis'].map(collect_map)
    data.drop(data.columns[[-1, 0]], axis=1, inplace=True)

    # put features & outputs in different DataFrames for convenience
    Y = data.loc[:, 'diagnosis']  # all rows of 'diagnosis'
    X = data.iloc[:, 1:]  # all rows of column 1 and ahead (features)

    # dividing the dataset into train and test sets
    # it is our dataframes
    # tts(*arrays, test_size, train_size=None, train_size, random_state=None, shuffle=True, stratify=None)
    X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.1, train_size=0.9, random_state=42)
    # W = sgd(X_train.to_numpy(), y_train.to_numpy())
    # weights array of given shape and type, filled with zeros.
    # shape(n) - return shape of the array 
    weights = np.zeros(X_train.shape[1])
    cost = svm(weights, features, outputs)
    print("Cost is: {}".format(cost))
    print("training finished.")
    print("weights are: {}".format(W))


def sgd(features, outputs):
    # Stop training once max number of epochs is reached. In our case is 5000
    max_epochs = 5000
    # weights array of given shape and type, filled with zeros.
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)
        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is:{} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights

In [ ]:
#Sotmax Implementation
#To avoid this problem, we normalize each value  by subtracting the largest value.
def softmax(z):
    z -= np.max(z)
    return np.exp(z) / np.sum(np.exp(z))

In [ ]:
def ridge_regression(X, y, alpha=0.01, lambda_value=1, epochs=30):
 
    m = np.shape(X)[0]  # total number of samples
    n = np.shape(X)[1]  # total number of features
 
    X = np.concatenate((np.ones((m, 1)), X), axis=1)
    W = np.random.randn(n + 1, )
 
    # stores the updates on the cost function (loss function)
    cost_history_list = []
 
    # iterate until the maximum number of epochs
    for current_iteration in range(epochs):  # begin the process
 
        # compute the dot product between our feature 'X' and weight 'W'
        y_estimated = X.dot(W)
 
        error = y_estimated - y
 
        ridge_reg_term = (lambda_value / 2 * m) * np.sum(np.square(W))
 
        cost = (1 / 2 * m) * np.sum(error ** 2) + ridge_reg_term
 
        gradient = (1 / m) * (X.T.dot(error) + (lambda_value * W))
 
        W = W - alpha * gradient
 
        print(f"cost:{cost} \t iteration: {current_iteration}")
 
        cost_history_list.append(cost)
 
    return W, cost_history_list